In [ ]:
import json
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# Твоя проверенная логика заголовков
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def chunk_batch(batch):
    all_chunks = []
    all_parent_ids = []
    all_metadata = []

    for i in range(len(batch['markdown'])):
        doc_id = batch['doc_id'][i]
        text = batch['markdown'][i]
        
        # 1. Сплит по заголовкам
        header_splits = header_splitter.split_text(text)
        
        # 2. Рекурсивный сплит
        final_splits = text_splitter.split_documents(header_splits)
        
        for doc in final_splits:
            all_chunks.append(doc.page_content)
            all_parent_ids.append(doc_id)
            # Сериализуем метаданные в строку, чтобы Parquet их "переварил"
            all_metadata.append(json.dumps(doc.metadata)) 

    return {
        "chunk_text": all_chunks, 
        "parent_doc_id": all_parent_ids,
        "section_metadata": all_metadata
    }

In [ ]:
import os
from datasets import load_dataset

# 1. Загрузка данных из обработанного Parquet
# Ты упомянул, что работаешь с Yandex Object Storage и S3, 
# поэтому путь должен указывать на локальную копию, синхронизированную через DVC
ds = load_dataset("parquet", data_files="../data/processed/texts.parquet", split="train")

# 2. Запуск чанкования
# Благодаря выносу в .py файл, это будет летать на всех ядрах без ошибок импорта
chunked_ds = ds.map(
    chunk_batch,
    batched=True,
    num_proc=os.cpu_count(),
    remove_columns=ds.column_names
)

# 3. Сохранение финальных чанков
# Эти данные пойдут на Runpod для генерации эмбеддингов
chunked_ds.to_parquet("../data/processed/chunks.parquet")

print(f"Подготовлено чанков: {len(chunked_ds)}")